##Import

In [2]:
import os
os.environ["OMP_NUM_THREADS"] = "8"        # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "8"   # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "8"        # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "8" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "8"

In [3]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
#import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import trange,tqdm
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
import implicit
import pickle
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype
from typing import List
import torch
import torch.nn.functional as F
from torch import nn
from polara.evaluation.pipelines import random_grid
import bottleneck as bn
from typing import List

In [4]:
import sys
sys.path.append('/home/oolaleke/incremental_PSI/Scripts/')

In [5]:
import a_DataHelpers
import b_AlgoFunctions
import c_PredEval
import d_CorrScores
import e_VAEfunction

In [6]:
from a_DataHelpers import getPivotMonths,Time_DataSplit,TestTrain_DataSplit,SingleRatingMatrix
from a_DataHelpers import AllRatingMatrices,All_SingleStepRatMat,psiStep_RatMat,psiAllStep_RatMat
from a_DataHelpers import Find_NewUsersItems,adjustedPSI_DF,adjustedAllDF,get_NEWHoldout,ADJUST_mainDF
from a_DataHelpers import  SingleStepRatMat_2,AllSingleStepRatMat_2,SingleStep_RatMat

from b_AlgoFunctions import integrator,getStartingValues,integratorOnMat,last_psiTrainMat, pureSVD
from b_AlgoFunctions import Updt_RowMatrix,getRow_Mat,row_update, Updt_ColMatrix,getCol_Mat,colunm_update
from b_AlgoFunctions import UsersItems_RatPair,getRowCol_psiupdt,ITEMS_defferredStatus,USERS_defferredStatus
from b_AlgoFunctions import getV_listUpdate, SingleStep_UPDATE,ALLSTEPs_UPDATE
from b_AlgoFunctions import tune_ALS,auc_score,TrainTest,mean_auc
from b_AlgoFunctions import nonzeros,least_squares_cg,alternating_least_squares_cg


from c_PredEval import TopNPred,TQDMgetALLTopNPred, Hitrate_Eval,getAll_HitRate,mean_confidence_interval
from c_PredEval import TQDMgetALLTopNPred_ALLUSERS,getALLTopNPred_ALLUSERS,get_ALLRandPred
from c_PredEval import topN_Index,getAll_RandomHitRate,getMOSTPOP_Pred,getAll_MostPOPHitRate,getAll_MOSTPOP_Pred
from c_PredEval import StepCoverage_Ratio,AllSteps_Coverage_Ratio,getAll_VAEMRR
from c_PredEval import MRR_Eval,getAll_MRR_Eval,TQDMgetAll_MRR_Eval, getAll_MostPOP_MRR,getAll_RandomRecMRR
from c_PredEval import ials_TopNPred, ials_getALLPred, ials_getALLPredTQDM,ialsALLPred_ALLUSERS

from d_CorrScores import no_copy_csr_matrix,build_rank_weights_matrix,rank_weighted_jaccard_index
from d_CorrScores import Updt_getAll_AvgCorr, updtCorr_4AllRanks

from e_VAEfunction import split_users, split_train_test, numerize_data, get_TrainRatMat, getTrainTest_RatMat

# Functions

##Multi VAE Model

In [7]:
class Encoder(nn.Module):
    def __init__(self, q_dims, dropout):  ## q_dims: List[int], dropout: List[float]
        super().__init__()

        self.q_dims = q_dims
        q_dims_ = self.q_dims[:-1] + [self.q_dims[-1] * 2]
        self.q_layers = nn.Sequential()
        for i, (p, inp, out) in enumerate(zip(dropout, q_dims_[:-1], q_dims_[1:])):
            self.q_layers.add_module("_".join(["dropout", str(i)]), nn.Dropout(p))
            self.q_layers.add_module("_".join(["linear", str(i)]), nn.Linear(inp, out))

    def forward(self, X):
        h = F.normalize(X, p=2, dim=1)
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = torch.tanh(h)
            else:
                mu, logvar = torch.split(h, self.q_dims[-1], dim=1)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, p_dims, dropout):  #p_dims: List[int], dropout: List[float]
        super().__init__()

        self.p_layers = nn.Sequential()
        for i, (p, inp, out) in enumerate(zip(dropout, p_dims[:-1], p_dims[1:])):
            self.p_layers.add_module("_".join(["dropout", str(i)]), nn.Dropout(p))
            self.p_layers.add_module("_".join(["linear", str(i)]), nn.Linear(inp, out))

    def forward(self, X):
        h = X
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = torch.tanh(h)
        return h


class MultiVAE(nn.Module):
    def __init__(self,p_dims,q_dims,dropout_enc,dropout_dec): #p_dims:List[int],q_dims:List[int],dropout_enc:List[float],dropout_dec:List[float], 
        super().__init__()
        self.encode = Encoder(q_dims, dropout_enc)  #q_dims == Encoder dimensions
        self.decode = Decoder(p_dims, dropout_dec)  #p_dims == Decoder dimensions

    def forward(self, X):
        mu, logvar = self.encode(X)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        sampled_z = mu + float(self.training) * eps * std
        return self.decode(sampled_z), mu, logvar        

def init_weights(model):
    for name, param in model.named_parameters():
        if "weight" in name:
            nn.init.xavier_uniform_(param.data)
        elif "bias" in name:
            param.data.normal_(std=0.001)
  

def getTot_annealSTeps(train_RatMat,batch_size,n_epochs,anneal_cap):
    training_steps = len(range(0, train_RatMat.shape[0], batch_size))
    try:
       total_anneal_steps = (training_steps * (n_epochs - int(n_epochs * 0.2))) / anneal_cap  ##
    except ZeroDivisionError:
        assert (constant_anneal), "if 'anneal_cap' is set to 0.0 'constant_anneal' must be set to 'True"

    return total_anneal_steps

## Train and Eval

In [8]:
def train_step(model, optimizer, data, epoch):
    model.train()
    running_loss = 0.0
    global update_count
    N = data.shape[0]
    idxlist = list(range(N))
    np.random.shuffle(idxlist)
    training_steps = len(range(0, N, batch_size))   #training_steps
    for batch_idx, start_idx in zip(range(training_steps), range(0, N, batch_size)):
        end_idx = min(start_idx + batch_size, N)
        X_inp = data[idxlist[start_idx:end_idx]]
        X_inp = torch.FloatTensor(X_inp.toarray()).to(device)
        if constant_anneal:
            anneal = anneal_cap
        else:
            anneal = min(anneal_cap, update_count / total_anneal_steps)
        update_count += 1
        optimizer.zero_grad()
        X_out, mu, logvar = model(X_inp)
        loss = vae_loss_fn(X_inp, X_out, mu, logvar, anneal)
        train_step.anneal = anneal
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        avg_loss = running_loss / (batch_idx + 1)
        #t.set_postfix(loss=avg_loss)


def eval_step(model,data_tr):
    model.eval()
    running_loss = 0.0
    eval_idxlist = list(range(data_tr.shape[0]))
    eval_N = data_tr.shape[0]
    eval_steps = len(range(0, eval_N, batch_size))
    n10_list,r10_list,hr10_list,mrr10_list = [], [], [], []
    with torch.no_grad():
         for batch_idx, start_idx in zip(range(eval_steps), range(0, eval_N, batch_size)):
             end_idx = min(start_idx + batch_size, eval_N)
             X_tr = data_tr[eval_idxlist[start_idx:end_idx]]
             #X_te = data_te[eval_idxlist[start_idx:end_idx]]
             X_tr_inp = torch.FloatTensor(X_tr.toarray()).to(device)

             X_out, mu, logvar = model(X_tr_inp)
             X_out = X_out.cpu().numpy()                           #X_out[X_tr.nonzero()] = -np.inf  
               
             n10 =  NDCG_binary_at_k_batch(X_out, X_tr, k=10)
             r10 =  Recall_at_k_batch(X_out, X_tr, k=10)
             hr10 =  HR_at_k_batch(X_out, X_tr.toarray(), k=10)     #X_tr.toarray() == groundTruth
             mrr10 = MRR_at_k_batch(X_out, X_tr.toarray(), k=10)

             n10_list.append(n10)
             r10_list.append(r10)
             hr10_list.append(hr10)
             mrr10_list.append(mrr10)
                
    n10_list = np.concatenate(n10_list)
    r10_list = np.concatenate(r10_list)
    hr10_list = np.concatenate(hr10_list)
    mrr10_list = np.concatenate(mrr10_list)
    return np.mean(hr10_list), np.mean(mrr10_list),np.mean(r10_list), np.mean(n10_list)



## Loss & Metric

In [9]:
def vae_loss_fn(inp, out, mu, logvar, anneal):
    neg_ll = -torch.mean(torch.sum(F.log_softmax(out, 1) * inp, -1))
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
    return neg_ll + anneal * KLD


In [10]:
def topN_Index(a, n):
    parted = np.argpartition(a, -n)[-n:]
    return parted[np.argsort(-a[parted])]  


def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=10):  #Hello2021
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]

    # build the discount template
    tp = 1.0 / np.log2(np.arange(2, k + 2))
    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis], idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[: min(n, k)]).sum() for n in heldout_batch.getnnz(axis=1)])
    return DCG[IDCG > 0.0] / IDCG[IDCG > 0.0]


def Recall_at_k_batch(X_pred, heldout_batch, k=10):
    batch_users = X_pred.shape[0]
    
    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(np.float32)
    denominator = np.minimum(k, X_true_binary.sum(axis=1))
    recall = tmp[denominator > 0.0] / denominator[denominator > 0.0]
    return recall

def HR_at_k_batch(pred_scores, ground_truth, k=10):  #"pred_scores' and 'ground_truth' must have the same shape."
    k = min(pred_scores.shape[1], k)
    idx = bn.argpartition(-pred_scores, k-1, axis=1)
    pred_scores_binary = np.zeros_like(pred_scores, dtype=bool)
    pred_scores_binary[np.arange(pred_scores.shape[0])[:, np.newaxis], idx[:, :k]] = True
    X_true_binary = (ground_truth > 0)
    num = (np.logical_and(X_true_binary, pred_scores_binary).sum(axis=1)).astype(np.float32)
    return num > 0
   
def MRR_at_k_batch(pred_scores, ground_truth, k=10):  #'pred_scores' and 'ground_truth' must have the same shape
       #Compute the Mean Reciprocal Rank (MRR)             
    k = min(pred_scores.shape[1], k)
    idx = np.argsort(-pred_scores)
    hits = ground_truth[np.arange(ground_truth.shape[0])[:, np.newaxis], idx[:, :k]]
    rranks, cranks = hits.nonzero()

    mrr = [0. for _ in range(ground_truth.shape[0])]
    for i, r in enumerate(rranks):
        if mrr[r] == 0:
           mrr[r] = 1. / (1 + cranks[i])

    return np.array(mrr)


## Get Pred & Metrics

In [11]:
##Generate Prediction list:
def topN_Index(a, n):
    parted = np.argpartition(a, -n)[-n:]
    return parted[np.argsort(-a[parted])]  

   #getmetrics_(model_,holdout_Mat,holdout_df,batch_size=holdout_Mat.shape[0],k)
def getmetrics_(model,data_tr,holdout_df,k=10):
    model.eval()
    running_loss = 0.0
    eval_idxlist = list(range(data_tr.shape[0]))
    eval_N = data_tr.shape[0]
    eval_steps = len(range(0, eval_N, batch_size))
    n100_list, hr10_list, mrr10_list = [], [], []
    users_pred = []
    with torch.no_grad():
         for batch_idx, start_idx in zip(range(eval_steps), range(0, eval_N, batch_size)):
             end_idx = min(start_idx + batch_size, eval_N)
             X_tr = data_tr[eval_idxlist[start_idx:end_idx]]
             X_tr_inp = torch.FloatTensor(X_tr.toarray()).to(device)
             #Make pred:    
             X_out, mu, logvar = model(X_tr_inp)
             X_out = X_out.cpu().numpy() 
             X_out[X_tr.nonzero()] = -np.inf        
             topN_pred = np.apply_along_axis(topN_Index, 1, X_out,n = 10)     
             users_pred.append(topN_pred)        
         AllUserPRED = np.vstack(users_pred)
  
    test_users = holdout_df['Updated_UserID'].unique()
    TestUserPRED =  AllUserPRED[test_users,:]    
    hr_value = Hitrate_Eval(holdout_df,TestUserPRED,'Updated_UserID','Updated_ItemID')   
    mrr_value = MRR_Eval(holdout_df,TestUserPRED,'Updated_ItemID') 
      
    return AllUserPRED, hr_value, mrr_value 


In [12]:
##Tuning......
def HitrateEval_(Holdout,TopN_pred,user_column,item_column):
    Eval_itemsVector  =  Holdout[[item_column]].to_numpy()
    HitRate_arr   =  (TopN_pred == Eval_itemsVector).sum(axis=1)  ##sum along row...
    HitCount = np.count_nonzero(HitRate_arr == 1)
    HitRate_ = HitRate_arr.mean()
    return HitRate_

def getEvalmetrics_(model,data_tr,holdout_df,k=10):
    model.eval()
    running_loss = 0.0
    eval_idxlist = list(range(data_tr.shape[0]))
    eval_N = data_tr.shape[0]
    eval_steps = len(range(0, eval_N, batch_size))
    hr10_list, mrr10_list = [], []
    users_pred = []
    with torch.no_grad():
         for batch_idx, start_idx in zip(range(eval_steps), range(0, eval_N, batch_size)):
             end_idx = min(start_idx + batch_size, eval_N)
             X_tr = data_tr[eval_idxlist[start_idx:end_idx]]
             X_tr_inp = torch.FloatTensor(X_tr.toarray()).to(device)
             #Make pred:    
             X_out, mu, logvar = model(X_tr_inp)
             X_out = X_out.cpu().numpy() 
             X_out[X_tr.nonzero()] = -np.inf        
             topN_pred = np.apply_along_axis(topN_Index, 1, X_out,n = 10)     
             users_pred.append(topN_pred)        
         AllUserPRED = np.vstack(users_pred)
  
    test_users = holdout_df['Updated_UserID'].unique()
    TestUserPRED =  AllUserPRED[test_users,:]    
    hr_value  = HitrateEval_(holdout_df,TestUserPRED,'Updated_UserID','Updated_ItemID')   
    mrr_value = MRR_Eval(holdout_df,TestUserPRED,'Updated_ItemID')      
    return hr_value, mrr_value 

def getModel(train_Mat,lr_):
    I =  train_Mat.shape[1] 
    q_dims = [I] + [600, 200]
    p_dims = [200, 600] + [I]
    dropout_enc = [0.5, 0.]
    dropout_dec = [0., 0.]

    VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)
    init_weights(VAEmodel)
    VAEmodel.to(device)
    optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=lr_, weight_decay=0.0)  
    return VAEmodel, optimizer   


def tuning_model(train_Mat,holdout_df,als_param_grid):
    best_hr = -np.inf  
    for i in trange(len(als_param_grid)): 
        batch_size, lr, n_epochs = list(als_param_grid)[i]  
        model, optimizer = getModel(train_Mat,lr)
        total_anneal_steps = getTot_annealSTeps(train_Mat,batch_size,n_epochs,anneal_cap)
        for epoch in range(n_epochs):
            train_step(model, optimizer, train_Mat, epoch)

        hr_, mrr_ = getEvalmetrics_(model,train_Mat,holdout_df,k=10)
        print('\n BS_ = {} | lr_ = {} | n_epoch = {}| HR = {} | MRR = {} '.format(batch_size, lr, n_epochs, hr_, mrr_))
        if (hr_ > best_hr):
           best_BS  = batch_size
           best_lr = lr
           best_epoch = n_epochs
           best_hr = hr_
           best_mrr = mrr_
               
    print('\n Best_param; BS_ = {} | lr_ = {} | n_epoch = {} | HR = {} | MRR = {}'.format(best_BS,best_lr,best_epoch,best_hr,best_mrr))
    return best_BS,best_lr,best_epoch,best_hr,best_mrr
               

In [35]:
def getVAEModel(train_Mat,lr_,p_dims):
    I =  train_Mat.shape[1] 
    q_Dims = [I] + p_dims[::-1]
    P_Dims = p_dims + [I]
    dropout_enc = [0.5, 0.]
    dropout_dec = [0., 0]

    VAEmodel = MultiVAE(P_Dims,q_Dims,dropout_enc,dropout_dec)
    init_weights(VAEmodel)
    VAEmodel.to(device)
    optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=lr_, weight_decay=0.0) 
    return VAEmodel, optimizer


def tuning_dim(train_Mat,holdout_df,pdim_list,lr,batch_size,n_epochs):
    best_hr = -np.inf  
    for p_dims in tqdm(pdim_list): 
        model, optimizer = getVAEModel(train_Mat,lr,p_dims)
        total_anneal_steps = getTot_annealSTeps(train_Mat,batch_size,n_epochs,anneal_cap)
        for epoch in range(n_epochs):
            train_step(model, optimizer, train_Mat, epoch)

        hr_, mrr_ = getEvalmetrics_(model,train_Mat,holdout_df,k=10)
        print('\n p_dim = {} ,HR = {} , MRR = {} '.format(p_dims,hr_,mrr_))
        if (hr_ > best_hr):
           best_hr = hr_
           best_mrr = mrr_
           best_dims = p_dims
               
    print('\n Best_param; p_dim = {},HR = {} ,MRR = {}'.format(best_dims,best_hr,best_mrr))
    return best_dims,best_hr,best_mrr
               

In [36]:
def singlestep_predEval(model,UserItemMat,holdout_df,n_epoch):
    for epoch in range(n_epoch):
        train_step(model, optimizer, UserItemMat, epoch)
  
    allUsersPRED, hr10,mrr10 = getmetrics_(model,UserItemMat,holdout_df,k=10) 
    return allUsersPRED, hr10 ,mrr10

def Allsteps_predEval(model,UserItemMatList,holdout_DFList,n_epoch):   
    hr10_List = [] 
    mrr10_List = [] 
    ALLUsers_Pred = []
    for UserItemMat,holdout_df in tqdm(zip(UserItemMatList,holdout_DFList)):
        allUsersPRED, hr10 ,mrr10= singlestep_predEval(model,UserItemMat,holdout_df,n_epoch)
        hr10_List.append(hr10)   
        mrr10_List.append(mrr10)
        ALLUsers_Pred.append(allUsersPRED)
        print("| HitRate  {:4.3f} | Mrr {:4.3f} | ".format(hr10, mrr10))  
    return hr10_List,mrr10_List ,ALLUsers_Pred 

In [14]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

## Import Data || ML_20M

In [14]:
#MOVIElen_20M
newMLDF_20M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_afterUpdt.csv.zip')
print(newMLDF_20M.shape) #userId_int
newMLDF_20M.head()

(9990682, 9)


,userid,movieId,rating,timestamp,timestamp_YM,userId,productId,Updated_UserID,Updated_ItemID
0,28507,1176,1,1995-01-09 11:46:44,1995-01,0,0,0,0
1,131160,47,1,1995-01-09 11:46:49,1995-01,1,1,1,1
2,85252,70,1,1996-01-29 00:00:00,1996-01,2,2,2,2
3,134445,45,1,1996-01-29 00:00:00,1996-01,3,3,3,3
4,99851,10,1,1996-01-29 00:00:00,1996-01,4,4,4,4


In [75]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)
    
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newUserItem.pkl', 'rb') as f:
     newUserItem_list = pickle.load(f)


In [16]:
UserItem_1 = newUserItem_list[0]
valid_1 = newHoldout_list[0]

print(UserItem_1['Updated_UserID'].max())
print(valid_1['Updated_UserID'].max())
print()
print(UserItem_1['Updated_ItemID'].max())
print(valid_1['Updated_ItemID'].max())

122396
122396

15297
15276


In [17]:
row_i =  UserItem_1['Updated_UserID'].max()+1
col_i =  UserItem_1['Updated_ItemID'].max()+1
TrainDFRatMat_ =  SingleRatingMatrix(UserItem_1,'Updated_UserID','Updated_ItemID',row_i,col_i)
TrainDFRatMat_

<122397x15298 sparse matrix of type '<class 'numpy.float64'>'
	with 8865107 stored elements in Compressed Sparse Row format>

In [18]:
ValidDFRatMat_ =  SingleRatingMatrix(valid_1,'Updated_UserID','Updated_ItemID',row_i,col_i)
ValidDFRatMat_

<122397x15298 sparse matrix of type '<class 'numpy.float64'>'
	with 3459 stored elements in Compressed Sparse Row format>

In [ ]:
##SVD Test..

In [32]:
def TopNPred_2(RatingMat,holdout,V, user_column, N):  ##N == Top_N
    TestUsers = holdout[user_column]
    HOLDOUT_usersMat = RatingMat[TestUsers,:]         ##this doubles as the "previously seen items"
    PVVT =  HOLDOUT_usersMat.dot(V).dot(V.T) 
    #users_column = HOLDOUT_usersMat.nonzero()[0]
    #items_column = HOLDOUT_usersMat.nonzero()[1]
    #args = np.array([users_column,items_column])
    #np.put(PVVT, np.ravel_multi_index(args, PVVT.shape),-np.inf)   ##downsample previously seen items
    TopN_pred = np.apply_along_axis(topN_Index, 1,PVVT,n = N)
    return TopN_pred

In [ ]:
TrainDFRatMat_

In [29]:
Usvd, Ssvd, VTsvd = svds(TrainDFRatMat_, k=50)
Vsvd = VTsvd.T 
print(Usvd.shape)
print(Vsvd.shape)

(122397, 50)
(15298, 50)


In [30]:
Topk_pred  =TopNPred(TrainDFRatMat_,valid_1,Vsvd,'Updated_UserID', N=10)
print(Topk_pred.shape)
Topk_pred

(3459, 10)


array([[4744, 2560, 1072, ..., 1069, 1070,  829],
       [ 980, 4652, 2250, ..., 4061, 1624, 1911],
       [2180,  532, 6381, ..., 1466, 3407,    1],
       ...,
       [ 180,  352,  179, ..., 1909,  969, 1917],
       [ 144, 1109, 1115, ..., 2542, 2501,  198],
       [2469, 1460,  157, ..., 2340, 2630, 2502]])

In [33]:
Topkpred_2  =TopNPred_2(TrainDFRatMat_,valid_1,Vsvd,'Updated_UserID', N=10)
print(Topkpred_2.shape)
Topkpred_2

(3459, 10)


array([[ 268,   76,  135, ..., 4137,   89, 4652],
       [1003,   20,  484, ...,  825,   58, 4137],
       [ 135,  268, 1062, ...,  576, 1052, 4724],
       ...,
       [ 180,  352,  179, ..., 1909,  969, 1917],
       [  92, 1053, 2707, ..., 1109, 1793, 1115],
       [2501, 2469, 1460, ..., 1793, 2340, 2630]])

In [31]:
Hitrate_Eval(valid_1,Topk_pred,'Updated_UserID','Updated_ItemID')  ##With -np.inf(removing seen items)

Number of hits:  205
Total Num of users:  3459
Recommendation HitRate:  0.059265683723619546


0.059265683723619546

In [34]:
Hitrate_Eval(valid_1,Topkpred_2,'Updated_UserID','Updated_ItemID')  ##Without -np.inf(including seen items)

Number of hits:  73
Total Num of users:  3459
Recommendation HitRate:  0.021104365423532813


0.021104365423532813

## Training

In [77]:
#

In [21]:
I =  TrainDFRatMat_.shape[1] 

q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)


init_weights(VAEmodel)
VAEmodel.to(device)
optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel              

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=15298, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=15298, bias=True)
    )
  )
)

In [22]:
batch_size = 500
anneal_cap = 0.2
update_count = 0
n_epochs = 200
constant_anneal = False
total_anneal_steps = 200000

In [24]:
for epoch in tqdm(range(n_epochs)):
    train_step(VAEmodel, optimizer, TrainDFRatMat_, epoch)


100%|██████████| 200/200 [27:00<00:00,  8.10s/it]


## Evaluation

In [27]:
##Generate Prediction list:
holdout_df = newHoldout_list[0]
AllUserPRED, hr_, mrr_ = getmetrics_(VAEmodel,TrainDFRatMat_,holdout_df,k=10)
print()
print("MRR   : ",mrr_)

Number of hits:  269
Total Num of users:  3459
Recommendation HitRate:  0.07776814108123735

MRR   :  0.028184584038877185


In [28]:
cov_ = StepCoverage_Ratio(newMLDF_20M,AllUserPRED,'Updated_ItemID')
cov_

0.26834

## Get AllStep Eval

In [57]:
rows_i = newMLDF_20M['Updated_UserID'].nunique()
cols_i = newMLDF_20M['Updated_ItemID'].nunique()
UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

15


[<136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8865107 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8945455 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9025842 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9106051 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9186201 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9266468 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9346597 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9426837 stored elements in Compres

In [61]:
I = newMLDF_20M['Updated_ItemID'].nunique()

q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)
init_weights(VAEmodel)
VAEmodel.to(device)
optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel              

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=20720, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=20720, bias=True)
    )
  )
)

In [60]:
batch_size = 500
anneal_cap = 0.2
update_count = 0
n_epochs = 200
constant_anneal = False

total_anneal_steps = 200000

In [62]:
                                       #Allsteps_predEval(model,UserItemMatList,holdout_DFList,n_epoch)
hr10_List,mrr10_List ,ALLUsers_Pred  = Allsteps_predEval(VAEmodel,UserItem_MATList,newHoldout_list,n_epochs)
print(hr10_List)
print(mrr10_List) 

1it [40:08, 2408.14s/it]Number of hits:  259
Total Num of users:  3459
Recommendation HitRate:  0.07487713211910957
| HitRate  0.075 | Mrr 0.029 | 
2it [1:20:17, 2408.85s/it]Number of hits:  202
Total Num of users:  3311
Recommendation HitRate:  0.06100875868317729
| HitRate  0.061 | Mrr 0.023 | 
3it [2:00:20, 2406.34s/it]Number of hits:  219
Total Num of users:  3124
Recommendation HitRate:  0.07010243277848911
| HitRate  0.070 | Mrr 0.026 | 
4it [2:40:18, 2402.86s/it]Number of hits:  215
Total Num of users:  3115
Recommendation HitRate:  0.06902086677367576
| HitRate  0.069 | Mrr 0.029 | 
5it [3:20:17, 2401.37s/it]Number of hits:  240
Total Num of users:  3165
Recommendation HitRate:  0.07582938388625593
| HitRate  0.076 | Mrr 0.026 | 
6it [4:00:16, 2400.76s/it]Number of hits:  178
Total Num of users:  3098
Recommendation HitRate:  0.057456423499031635
| HitRate  0.057 | Mrr 0.021 | 
7it [4:40:18, 2400.96s/it]Number of hits:  175
Total Num of users:  3169
Recommendation HitRate:  0.0

In [64]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML20mVAE_allPRED_NEW2.pkl', 'wb') as f:
     pickle.dump(ALLUsers_Pred, f)

In [65]:
hr10_List

[0.07487713211910957,
 0.06100875868317729,
 0.07010243277848911,
 0.06902086677367576,
 0.07582938388625593,
 0.057456423499031635,
 0.0552224676554118,
 0.0728667305848514,
 0.07894736842105263,
 0.0753886770313875,
 0.07621671258034894,
 0.07214819629509263,
 0.07756132756132757,
 0.0737518910741301,
 0.07838827838827839]

In [66]:
mrr10_List

[0.02933777539154816,
 0.023010240036818112,
 0.026060402820966198,
 0.028608244796045762,
 0.02621705659620352,
 0.021436000696814124,
 0.017781885027072782,
 0.028112714138601003,
 0.029159331861399528,
 0.028052726908696406,
 0.028610418761933914,
 0.028289124121103328,
 0.029036968322682613,
 0.030437288379799733,
 0.02793127507413222]

In [67]:
Cov_List = AllSteps_Coverage_Ratio(newMLDF_20M,ALLUsers_Pred,'Updated_ItemID')
Cov_List

[0.264913,
 0.258784,
 0.2639,
 0.259797,
 0.26221,
 0.261921,
 0.269064,
 0.2764,
 0.279585,
 0.276255,
 0.291409,
 0.295849,
 0.303909,
 0.311824,
 0.311004]

In [69]:
AllSteps = list(range(1,15))
print(len(AllSteps))
AllSteps

14


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [70]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(ALLUsers_Pred,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_

array([array([0.36369515, 0.54624489, 0.61474513, ..., 0.11059284, 0.50909834,
              0.56112521])                                                    ,
       array([0.35980103, 0.57142857, 0.64828048, ..., 0.37435993, 0.64387528,
              0.4640484 ])                                                    ,
       array([0.31980331, 0.84755945, 0.39659413, ..., 0.41737878, 0.1527409 ,
              0.04457968])                                                    ,
       array([0.35369097, 0.92514345, 0.42449098, ..., 0.        , 0.01736733,
              0.        ])                                                    ,
       array([0.42696955, 0.6221978 , 0.47738191, ..., 0.11411321, 0.35980103,
              0.24342992])                                                    ,
       array([0.33303233, 0.68458291, 0.44753873, ..., 0.37576887, 0.34505695,
              0.01933435])                                                    ,
       array([0.29582163, 0.44160156, 0.256126

In [71]:
UpdtPSICorr_.shape
for df in UpdtPSICorr_:
    print(df.mean())  

0.4890564348504757
0.5139198543271317
0.5110664391439165
0.5144665571911105
0.5099897313510342
0.5236193810611822
0.5211199667647584
0.5099962481868815
0.507363795405602
0.5180855235610252
0.520402266399777
0.5126099920020206
0.49874961971866727
0.4971462370271155


# Amazon Games

In [22]:
newAMZGames_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_afterUpdt.csv')
print(newAMZGames_DF.shape)  
newAMZGames_DF.head()

(345203, 9)


,userid,ProductID,rating,timestamp,timestamp_YM,userId,productId,Updated_UserID,Updated_ItemID
0,A2AXQTB83VMK4L,B0000296O5,1,1999-10-14,1999-10,0,0,0,0
1,A2T04VAIXSKJH2,B00002NDRY,1,1999-11-05,1999-11,1,1,1,1
2,A1QA8K3LD9K892,B000021Y5F,1,1999-11-10,1999-11,2,2,2,2
3,A1QA8K3LD9K892,B000021Y5Q,1,1999-11-10,1999-11,2,3,2,3
4,ANLA598UNJI8A,B00001IVGE,1,1999-11-14,1999-11,3,4,3,4


In [98]:
##lOAD Saved Lists...
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newUserItem.pkl', 'rb') as f:   #
     newUserItem_list = pickle.load(f)

with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)


In [25]:
UserItem_1 = newUserItem_list[0]
valid_1 = newHoldout_list[0]
print(UserItem_1['Updated_UserID'].max())
print(valid_1['Updated_UserID'].max())
print()
print(UserItem_1['Updated_ItemID'].max())
print(valid_1['Updated_ItemID'].max())

34575
34575

16138
16137


In [26]:
row_i =  UserItem_1['Updated_UserID'].max()+1
col_i =  UserItem_1['Updated_ItemID'].max()+1
TrainDFRatMat_1 =  SingleRatingMatrix(UserItem_1,'Updated_UserID','Updated_ItemID',row_i,col_i)
TrainDFRatMat_1

<34576x16139 sparse matrix of type '<class 'numpy.float64'>'
	with 250384 stored elements in Compressed Sparse Row format>

In [29]:
ValidDFRatMat_1 =  SingleRatingMatrix(valid_1,'Updated_UserID','Updated_ItemID',row_i,col_i)
ValidDFRatMat_1

<34576x16139 sparse matrix of type '<class 'numpy.float64'>'
	with 2018 stored elements in Compressed Sparse Row format>

In [79]:
I =  TrainDFRatMat_1.shape[1] 

q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]
VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel)
VAEmodel.to(device)
optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel              

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=16139, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=16139, bias=True)
    )
  )
)

In [105]:
batch_size = 200  #
anneal_cap = 0.2
update_count = 0
n_epochs = 100
constant_anneal = False

In [58]:
##Best parameters; BS_ = 100 ,lr_ = 0.001 , n_epoch = 50 

In [106]:
total_anneal_steps = getTot_annealSTeps(TrainDFRatMat_1,batch_size,n_epochs,anneal_cap)
total_anneal_steps

69200.0

In [73]:
for epoch in tqdm(range(n_epochs)):
    train_step(VAEmodel, optimizer, TrainDFRatMat_1, epoch)


100%|██████████| 100/100 [04:44<00:00,  2.84s/it]


In [74]:
#Generate Prediction list:
holdout_df1 = newHoldout_list[0]
AllUserPRED, hr_, mrr_ = getmetrics_(VAEmodel,TrainDFRatMat_1,holdout_df1,k=10)
print()
print("MRR   : ",mrr_)

Number of hits:  136
Total Num of users:  2018
Recommendation HitRate:  0.06739345887016848

MRR   :  0.02463385089747825


In [101]:
# for i,j in zip(newUserItem_list,newHoldout_list):
#     print(i['Updated_UserID'].max(),j['Updated_UserID'].max())
#     print(i['Updated_ItemID'].max(),j['Updated_ItemID'].max())
#     print()

In [102]:
rows_i = newAMZGames_DF['Updated_UserID'].nunique()
cols_i = newAMZGames_DF['Updated_ItemID'].nunique()

UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

15


[<39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 250384 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 255520 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 261105 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 266597 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 272102 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 277572 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 282899 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 288493 stored elements in Compressed Sparse Row f

In [110]:
batch_size = 200  #
anneal_cap = 0.2
update_count = 0
n_epochs = 100
constant_anneal = False

In [111]:
I = newAMZGames_DF['Updated_ItemID'].nunique()

q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel)
VAEmodel.to(device)
optimizer = torch.optim.AdamW(VAEmodel.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel   


MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=17318, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=17318, bias=True)
    )
  )
)

In [112]:
                                       #Allsteps_predEval(model,UserItemMatList,holdout_DFList,n_epoch)
hr10_List, mrr10_List, ALLUsers_Pred  = Allsteps_predEval(VAEmodel,UserItem_MATList,newHoldout_list,n_epochs) 
print(hr10_List)
print(mrr10_List) 

1it [05:51, 351.67s/it]Number of hits:  132
Total Num of users:  2018
Recommendation HitRate:  0.06541129831516353
| HitRate  0.065 | Mrr 0.026 | 
2it [11:41, 350.40s/it]Number of hits:  137
Total Num of users:  2200
Recommendation HitRate:  0.06227272727272727
| HitRate  0.062 | Mrr 0.028 | 
3it [17:35, 352.17s/it]Number of hits:  150
Total Num of users:  2172
Recommendation HitRate:  0.06906077348066299
| HitRate  0.069 | Mrr 0.029 | 
4it [23:27, 351.97s/it]Number of hits:  169
Total Num of users:  2079
Recommendation HitRate:  0.08128908128908129
| HitRate  0.081 | Mrr 0.038 | 
5it [29:19, 352.23s/it]Number of hits:  160
Total Num of users:  2089
Recommendation HitRate:  0.07659167065581618
| HitRate  0.077 | Mrr 0.034 | 
6it [35:13, 352.83s/it]Number of hits:  140
Total Num of users:  2120
Recommendation HitRate:  0.0660377358490566
| HitRate  0.066 | Mrr 0.035 | 
7it [41:05, 352.55s/it]Number of hits:  139
Total Num of users:  2217
Recommendation HitRate:  0.06269733874605322
| Hi

In [113]:
hr10_List

[0.06541129831516353,
 0.06227272727272727,
 0.06906077348066299,
 0.08128908128908129,
 0.07659167065581618,
 0.0660377358490566,
 0.06269733874605322,
 0.05663797009515179,
 0.07688701359587435,
 0.05849838783970521,
 0.054400707651481646,
 0.05832591273374889,
 0.04963570127504554,
 0.06478624944909651,
 0.050405040504050404]

In [114]:
mrr10_List

[0.02627050985574276,
 0.028140331890331886,
 0.02874517670788389,
 0.03762149079609397,
 0.03363593881784404,
 0.034990266546870326,
 0.030708951750900333,
 0.02722308527700462,
 0.03174621778700541,
 0.026930571092247738,
 0.023193838939084405,
 0.02433179267551485,
 0.024046426402983778,
 0.024205299866384044,
 0.021777892074921777]

In [116]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZgmsVAE_allPRED_NEW2.pkl', 'wb') as f: 
     pickle.dump(ALLUsers_Pred, f)

In [115]:
ALLUsers_Pred[:2]

[array([[  145,   243,   385, ...,   752,   204,    23],
        [   29,    28,  2612, ..., 14958,   681,  7492],
        [  151,   385,  1485, ...,   289,   225,   145],
        ...,
        [ 4190, 12918, 10874, ...,  4532,  7304, 12043],
        [ 4190, 12918, 10874, ...,  4532,  7304, 12043],
        [ 4190, 12918, 10874, ...,  4532,  7304, 12043]]),
 array([[  145,   243,   385, ...,   348,   436,  1050],
        [   29,  5867,  3525, ..., 14958,  9995,   681],
        [  385,   340,   145, ...,   270,   198,  1330],
        ...,
        [10874, 10344,  4190, ...,  9913, 15563,  4532],
        [10874, 10344,  4190, ...,  9913, 15563,  4532],
        [10874, 10344,  4190, ...,  9913, 15563,  4532]])]

In [117]:
Cov_List = AllSteps_Coverage_Ratio(newAMZGames_DF,ALLUsers_Pred,'Updated_ItemID')
Cov_List

[0.667976,
 0.682411,
 0.709955,
 0.723871,
 0.73438,
 0.737903,
 0.754995,
 0.763252,
 0.769777,
 0.766082,
 0.770874,
 0.774685,
 0.774512,
 0.782481,
 0.781499]

In [118]:
AllSteps = list(range(1,15))
print(len(AllSteps))
AllSteps

14


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [119]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(ALLUsers_Pred,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_


array([array([0.66708075, 0.32145734, 0.23366204, ..., 0.        , 0.        ,
              0.        ])                                                    ,
       array([0.42065249, 0.15058457, 0.11647255, ..., 0.        , 0.        ,
              0.        ])                                                    ,
       array([0.30915218, 0.30752879, 0.08880366, ..., 0.        , 0.01933435,
              0.02499653])                                                    ,
       array([0.25869714, 0.12326891, 0.28835748, ..., 0.08880366, 0.        ,
              0.21      ])                                                    ,
       array([0.27588591, 0.36143134, 0.09186391, ..., 0.        , 0.        ,
              0.02928462])                                                    ,
       array([0.29707407, 0.64497437, 0.43181377, ..., 0.04531936, 0.        ,
              0.        ])                                                    ,
       array([0.6166904 , 0.19559407, 0.237695

In [120]:
UpdtPSICorr_.shape
for df in UpdtPSICorr_:
    print(df.mean())  

0.3171733674250139
0.3424896252800678
0.35240674070693684
0.35635566796544155
0.3540354833310521
0.3585428097502578
0.3590566627369354
0.3575911089462821
0.35591817581563023
0.3502110547336786
0.35379508485915956
0.35726544280326017
0.35334324644624404
0.35097745703939615


In [130]:
# AllHitrate = SVDoptimalSearch(SVD_UserItemMAT[0],newHoldout_list[0],'Updated_UserID','Updated_ItemID',start=10,end=100,increment=10,N=10)

In [2]:
##MOVIELEN_1M

In [15]:
#MOVIElen_1M
newMLDF_1M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_afterUpdt.csv.zip')
newMLDF_1M['timestamp'] = pd.to_datetime(newMLDF_1M['timestamp']) 
print(newMLDF_1M.shape)   #userId_int
newMLDF_1M.head()

(575281, 9)


,userId_int,movieId_int,rating,timestamp,userId,productId,timestamp_YM,Updated_UserID,Updated_ItemID
0,6040,858,1,2000-04-25 23:05:32,0,0,2000-04,0,0
1,6040,2384,1,2000-04-25 23:05:54,0,1,2000-04,0,1
2,6040,593,1,2000-04-25 23:05:54,0,2,2000-04,0,2
3,6040,1961,1,2000-04-25 23:06:17,0,3,2000-04,0,3
4,6040,2019,1,2000-04-25 23:06:17,0,4,2000-04,0,4


In [16]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newUserItem.pkl', 'rb') as f:
     newUserItem_list = pickle.load(f)


In [17]:
UserItem_2 = newUserItem_list[0]
valid_2 = newHoldout_list[0]
print(UserItem_2['Updated_UserID'].max())
print(valid_2['Updated_UserID'].max())
print()
print(UserItem_2['Updated_ItemID'].max())
print(valid_2['Updated_ItemID'].max())


5787
5787

3483
3482


In [18]:
row_i =  UserItem_2['Updated_UserID'].max()+1
col_i =  UserItem_2['Updated_ItemID'].max()+1
TrainDFRatMat_2 =  SingleRatingMatrix(UserItem_2,'Updated_UserID','Updated_ItemID',row_i,col_i)
TrainDFRatMat_2

<5788x3484 sparse matrix of type '<class 'numpy.float64'>'
	with 500332 stored elements in Compressed Sparse Row format>

In [19]:
ValidDFRatMat_2 =  SingleRatingMatrix(valid_2,'Updated_UserID','Updated_ItemID',row_i,col_i)
ValidDFRatMat_2

<5788x3484 sparse matrix of type '<class 'numpy.float64'>'
	with 233 stored elements in Compressed Sparse Row format>

In [50]:
dim = [200, 600]    #[64,128]

In [51]:
I =  TrainDFRatMat_2.shape[1] 
q_dims = [I] + dim[::-1]   
p_dims = dim + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]
VAEmodel_ = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel_)
VAEmodel_.to(device)
optimizer = torch.optim.AdamW(VAEmodel_.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel_

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=3484, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=3484, bias=True)
    )
  )
)

In [52]:
#BS_ = 100 | lr_ = 0.001 | n_epoch = 50

batch_size = 100
anneal_cap = 0.2
update_count = 0
n_epochs = 50
constant_anneal = False

total_anneal_steps = getTot_annealSTeps(TrainDFRatMat_2,batch_size,n_epochs,anneal_cap)
total_anneal_steps


11600.0

In [53]:
for epoch in tqdm(range(n_epochs)):
    train_step(VAEmodel_, optimizer, TrainDFRatMat_2, epoch)

100%|██████████| 50/50 [00:16<00:00,  2.97it/s]


In [25]:
#Generate Prediction list:
holdout_df2 = newHoldout_list[0]  #BS=100, EPOCH =50  | [200,600]
AllUserPRED, hr_, mrr_ = getmetrics_(VAEmodel_,TrainDFRatMat_2,holdout_df2,k=10)
print()
print("MRR   : ",mrr_)

Number of hits:  20
Total Num of users:  233
Recommendation HitRate:  0.08583690987124463

MRR   :  0.019931534845697936


In [32]:
#Generate Prediction list:
holdout_df2 = newHoldout_list[0]  #BS=200, EPOCH =100
AllUserPRED, hr_, mrr_ = getmetrics_(VAEmodel_,TrainDFRatMat_2,holdout_df2,k=10)
print()
print("MRR   : ",mrr_)

Number of hits:  16
Total Num of users:  233
Recommendation HitRate:  0.06866952789699571

MRR   :  0.020234689011513044


In [48]:
als_params = dict(batch_size = [100,200,300,500],
                  lr_ = [1e-5,1e-3,1e-1],
                  n_epochs = [50,100,150,200])        # dim_i  = [0,1,2,3,4]
als_param_grid, als_param_names = random_grid(als_params, n=40)
print(als_param_names)
len(als_param_grid)  

('batch_size', 'lr_', 'n_epochs')


40

In [50]:
for i in range(10):
    batch_size, lr, n_epochs = list(als_param_grid)[i]  
    print(batch_size, lr, n_epochs)   #pdim_list[dim_i])

100 1e-05 50
100 0.001 200
200 0.001 50
300 1e-05 50
300 0.001 200
500 0.1 200
100 0.1 200
500 0.001 150
200 0.1 50
100 0.001 50


In [51]:
anneal_cap = 0.2
update_count = 0
constant_anneal = False    

                #tuning_model(train_Mat,holdout_df,als_param_grid)
bestModelInfo = tuning_model(TrainDFRatMat_2,holdout_df2,als_param_grid)

  2%|▎         | 1/40 [00:17<11:22, 17.49s/it]
 BS_ = 100 | lr_ = 1e-05 | n_epoch = 50| HR = 0.060085836909871244 | MRR = 0.022162613257033857 
  5%|▌         | 2/40 [01:24<29:26, 46.50s/it]
 BS_ = 100 | lr_ = 0.001 | n_epoch = 200| HR = 0.05150214592274678 | MRR = 0.014031950405340965 
  8%|▊         | 3/40 [01:40<20:16, 32.89s/it]
 BS_ = 200 | lr_ = 0.001 | n_epoch = 50| HR = 0.07296137339055794 | MRR = 0.02134341576401662 
 10%|█         | 4/40 [01:57<15:53, 26.49s/it]
 BS_ = 300 | lr_ = 1e-05 | n_epoch = 50| HR = 0.055793991416309016 | MRR = 0.01996900333810205 
 12%|█▎        | 5/40 [03:05<24:06, 41.34s/it]
 BS_ = 300 | lr_ = 0.001 | n_epoch = 200| HR = 0.04721030042918455 | MRR = 0.011008924313645344 
 15%|█▌        | 6/40 [04:14<28:42, 50.65s/it]
 BS_ = 500 | lr_ = 0.1 | n_epoch = 200| HR = 0.004291845493562232 | MRR = 0.0006131207847946045 
 18%|█▊        | 7/40 [05:20<30:42, 55.84s/it]
 BS_ = 100 | lr_ = 0.1 | n_epoch = 200| HR = 0.004291845493562232 | MRR = 0.0006131207847946

In [ ]:
##Tuning Dimension

In [38]:
pdim_list = [[32,64],[64,128],[256,512],[100,300],[200,600],[300,900]]

                             #tuning_dim(train_Mat,holdout_df,pdim_list,lr,batch_size,n_epochs):
best_dims,best_hr,best_mrr = tuning_dim(TrainDFRatMat_2,holdout_df2,pdim_list,lr=0.001,batch_size=100,n_epochs=50)

 17%|█▋        | 1/6 [00:09<00:45,  9.11s/it]
 p_dim = [32, 64] ,HR = 0.04291845493562232 , MRR = 0.01639757476667348 
 33%|███▎      | 2/6 [00:17<00:35,  8.93s/it]
 p_dim = [64, 128] ,HR = 0.06866952789699571 , MRR = 0.0155971115198583 
 50%|█████     | 3/6 [00:26<00:26,  8.94s/it]
 p_dim = [256, 512] ,HR = 0.034334763948497854 , MRR = 0.006764765992233803 
 67%|██████▋   | 4/6 [00:35<00:17,  8.96s/it]
 p_dim = [100, 300] ,HR = 0.04721030042918455 , MRR = 0.011587982832618027 
 83%|████████▎ | 5/6 [00:43<00:08,  8.63s/it]
 p_dim = [200, 600] ,HR = 0.06437768240343347 , MRR = 0.012224947203487975 
100%|██████████| 6/6 [00:52<00:00,  8.82s/it]
 p_dim = [300, 900] ,HR = 0.060085836909871244 , MRR = 0.02159377341780775 

 Best_param; p_dim = [64, 128],HR = 0.06866952789699571 ,MRR = 0.0155971115198583



In [72]:
rows_i = newMLDF_1M['Updated_UserID'].nunique()
cols_i = newMLDF_1M['Updated_ItemID'].nunique()
UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

10


[<6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 500332 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 508812 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 517278 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 525757 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 534098 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 542538 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 551048 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 559597 stored elements in Compressed Sparse Row format>,
 <6038x3

In [74]:
dim = [200,600]
I =  UserItem_MATList[0].shape[1] 
q_dims = [I] + dim[::-1]   
p_dims = dim + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]
VAEmodel_ml1 = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel_ml1)
VAEmodel_ml1.to(device)
optimizer = torch.optim.AdamW(VAEmodel_ml1.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel_ml1

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=3533, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=3533, bias=True)
    )
  )
)

In [82]:
#BS_ = 100 | lr_ = 0.001 | n_epoch = 50
                                    
batch_size = 100
anneal_cap = 0.2
update_count = 0
n_epochs = 50
constant_anneal = False

total_anneal_steps = getTot_annealSTeps(TrainDFRatMat_2,batch_size,n_epochs,anneal_cap)
total_anneal_steps

11600.0

In [62]:
                                       #BS_ = 100 | lr_ = 0.001 | n_epoch = 50
                                       #Allsteps_predEval(model,UserItemMatList,holdout_DFList,n_epoch)
hr10_List, mrr10_List, ALLUsers_Pred  = Allsteps_predEval(VAEmodel_ml1,UserItem_MATList,newHoldout_list,n_epochs) 
print(hr10_List)
print(mrr10_List) 

1it [00:18, 18.50s/it]Number of hits:  14
Total Num of users:  233
Recommendation HitRate:  0.060085836909871244
| HitRate  0.060 | Mrr 0.014 | 
2it [00:36, 17.94s/it]Number of hits:  17
Total Num of users:  255
Recommendation HitRate:  0.06666666666666667
| HitRate  0.067 | Mrr 0.028 | 
3it [00:52, 17.21s/it]Number of hits:  28
Total Num of users:  286
Recommendation HitRate:  0.0979020979020979
| HitRate  0.098 | Mrr 0.026 | 
4it [01:09, 17.03s/it]Number of hits:  21
Total Num of users:  309
Recommendation HitRate:  0.06796116504854369
| HitRate  0.068 | Mrr 0.027 | 
5it [01:27, 17.48s/it]Number of hits:  17
Total Num of users:  468
Recommendation HitRate:  0.03632478632478633
| HitRate  0.036 | Mrr 0.006 | 
6it [01:45, 17.66s/it]Number of hits:  28
Total Num of users:  526
Recommendation HitRate:  0.053231939163498096
| HitRate  0.053 | Mrr 0.017 | 
7it [02:03, 17.83s/it]Number of hits:  34
Total Num of users:  517
Recommendation HitRate:  0.06576402321083172
| HitRate  0.066 | Mrr 

In [63]:
hr10_List

[0.060085836909871244,
 0.06666666666666667,
 0.0979020979020979,
 0.06796116504854369,
 0.03632478632478633,
 0.053231939163498096,
 0.06576402321083172,
 0.04042553191489362,
 0.05739514348785872,
 0.049723756906077346]

In [64]:
mrr10_List

[0.014461134954697185,
 0.027972300031123564,
 0.025538350538350536,
 0.026945600246571118,
 0.0063025708859042184,
 0.01651804574808377,
 0.017650670842160203,
 0.014615839243498819,
 0.014880514383825642,
 0.014007278786284313]

In [65]:
ALLUsers_Pred[:2]

[array([[ 247,  566,  256, ..., 1564,  246,  134],
        [ 369,  563, 1650, ...,  630,  564,  398],
        [ 478,   73,   59, ...,   19,  138,  538],
        ...,
        [  19,   91,   22, ...,  330,  129,   89],
        [  19,   91,   22, ...,  330,  129,   89],
        [  19,   91,   22, ...,  330,  129,   89]]),
 array([[1042,  247,  566, ...,  134,  242, 1296],
        [1650,  538,  497, ...,  240,  412,  564],
        [ 478,  219,   54, ...,  564,  408,   73],
        ...,
        [  19,  137,   91, ...,  330,    2,  129],
        [  19,  137,   91, ...,  330,    2,  129],
        [  19,  137,   91, ...,  330,    2,  129]])]

In [66]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/ML_1MVAE_allPRED_NEW.pkl', 'wb') as f: 
     pickle.dump(ALLUsers_Pred, f)

In [67]:
Cov_List = AllSteps_Coverage_Ratio(newMLDF_1M,ALLUsers_Pred,'Updated_ItemID')
Cov_List    #Hello2021

[0.536088,
 0.49533,
 0.467025,
 0.457968,
 0.447495,
 0.43957,
 0.442966,
 0.431644,
 0.442117,
 0.438155]

In [68]:
AllSteps = list(range(1,10))
print(len(AllSteps))
AllSteps

9


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [70]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(ALLUsers_Pred,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_

array([array([0.32680209, 0.2472119 , 0.52326901, ..., 0.3441996 , 0.56443408,
              0.32216749])                                                    ,
       array([0.39067358, 0.20930614, 0.63931149, ..., 0.41833205, 0.05578601,
              0.65939748])                                                    ,
       array([0.38610329, 0.22151427, 0.71332405, ..., 0.37346483, 0.01933435,
              0.3952741 ])                                                    ,
       array([0.3243025 , 0.19967493, 0.56228172, ..., 0.20584872, 0.2293471 ,
              0.15310108])                                                    ,
       array([0.2405042 , 0.29309741, 0.40363221, ..., 0.22080715, 0.32264134,
              0.67902639])                                                    ,
       array([0.41874099, 0.41168595, 0.43557328, ..., 0.12773109, 0.29468514,
              0.15121266])                                                    ,
       array([0.25122902, 0.26441113, 0.697952

In [71]:
print(UpdtPSICorr_.shape)
for df in UpdtPSICorr_:
    print(df.mean())  

(9,)
0.3453725730844539
0.4009852104303515
0.43006594151081473
0.438709649557558
0.4388300808238541
0.4403475362482562
0.44581312224985903
0.44451138645394694
0.44270213323950186


# Amazon_Bty

In [131]:
newAMZB_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_afterUpdt.csv') 
newAMZB_DF['timestamp'] = pd.to_datetime(newAMZB_DF['timestamp']) 
print(newAMZB_DF.shape)   #userId_int
newAMZB_DF.head()

(51277, 7)


,userId,productId,rating,timestamp,timestamp_YM,Updated_UserID,Updated_ItemID
0,0,0,1,2001-06-08,2001-06,0,0
1,1,0,1,2002-06-12,2002-06,1,0
2,1,1,1,2002-06-12,2002-06,1,1
3,2,1,1,2002-06-17,2002-06,2,1
4,3,1,1,2002-07-04,2002-07,3,1


In [241]:

with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newHoldout.pkl', 'rb') as f:
     newHoldout_list = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newUserItem.pkl', 'rb') as f:
     newUserItem_list = pickle.load(f)


In [230]:
UserItem_2 = newUserItem_list[0]
valid_2 = newHoldout_list[0]
print(UserItem_2['Updated_UserID'].max())
print(valid_2['Updated_UserID'].max())
print()
print(UserItem_2['Updated_ItemID'].max())
print(valid_2['Updated_ItemID'].max())

19385
19385

6536
6530


In [138]:
row_i =  UserItem_2['Updated_UserID'].max()+1
col_i =  UserItem_2['Updated_ItemID'].max()+1
TrainDFRatMat_2 =  SingleRatingMatrix(UserItem_2,'Updated_UserID','Updated_ItemID',row_i,col_i)
TrainDFRatMat_2

<19386x6537 sparse matrix of type '<class 'numpy.float64'>'
	with 40228 stored elements in Compressed Sparse Row format>

In [139]:
ValidDFRatMat_2 =  SingleRatingMatrix(valid_2,'Updated_UserID','Updated_ItemID',row_i,col_i)
ValidDFRatMat_2

<19386x6537 sparse matrix of type '<class 'numpy.float64'>'
	with 474 stored elements in Compressed Sparse Row format>

In [211]:
I =  TrainDFRatMat_2.shape[1] 
q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]
VAEmodel_bty = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel_bty)
VAEmodel_bty.to(device)
optimizer = torch.optim.AdamW(VAEmodel_bty.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel_bty              

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=6537, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=6537, bias=True)
    )
  )
)

In [ ]:
#Best_param; BS_ = 200 | lr_ = 0.001 | n_epoch = 100 |

In [212]:
batch_size = 200
anneal_cap = 0.2
update_count = 0
n_epochs = 100
constant_anneal = False

total_anneal_steps = getTot_annealSTeps(TrainDFRatMat_2,batch_size,n_epochs,anneal_cap)
total_anneal_steps

38800.0

In [213]:
for epoch in tqdm(range(n_epochs)):
    train_step(VAEmodel_bty, optimizer, TrainDFRatMat_2, epoch)

100%|██████████| 100/100 [01:22<00:00,  1.22it/s]


In [214]:
#Generate Prediction list:
holdout_df2 = newHoldout_list[0]
AllUserPRED, hr_, mrr_ = getmetrics_(VAEmodel_bty,TrainDFRatMat_2,holdout_df2,k=10)
print()
print("MRR   : ",mrr_)

Number of hits:  162
Total Num of users:  474
Recommendation HitRate:  0.34177215189873417

MRR   :  0.30471753398968593


## Tuning

In [190]:
als_params = dict(batch_size = [100,200,300,500],
                  lr_ = [1e-5,1e-3,1e-1],
                  n_epochs = [50,100,150,200])        # dim_i  = [0,1,2,3,4]
als_param_grid, als_param_names = random_grid(als_params, n=40)
print(als_param_names)
len(als_param_grid)                 

('batch_size', 'lr_', 'n_epochs')


40

In [191]:
for i in range(10):
    batch_size, lr, n_epochs = list(als_param_grid)[i]  
    print(batch_size, lr, n_epochs)   #pdim_list[dim_i])

100 1e-05 50
200 0.1 200
100 0.001 200
200 0.001 50
300 1e-05 50
200 1e-05 150
300 0.001 200
500 0.1 200
100 0.1 200
500 0.001 150


In [202]:
anneal_cap = 0.2
update_count = 0
constant_anneal = False    

                 #tuning_model(train_Mat,holdout_df,als_param_grid)
bestModelInfo =  tuning_model(TrainDFRatMat_2,holdout_df2,als_param_grid)

  2%|▎         | 1/40 [00:28<18:21, 28.24s/it]
 BS_ = 100 | lr_ = 1e-05 | n_epoch = 50| HR = 0.1518987341772152 | MRR = 0.08615631906771147 
  5%|▌         | 2/40 [02:11<45:40, 72.11s/it]
 BS_ = 200 | lr_ = 0.1 | n_epoch = 200| HR = 0.0 | MRR = 0 
  8%|▊         | 3/40 [03:54<53:17, 86.41s/it]
 BS_ = 100 | lr_ = 0.001 | n_epoch = 200| HR = 0.3459915611814346 | MRR = 0.31348452883263006 
 10%|█         | 4/40 [04:22<37:57, 63.27s/it]
 BS_ = 200 | lr_ = 0.001 | n_epoch = 50| HR = 0.33544303797468356 | MRR = 0.3109821847163619 
 12%|█▎        | 5/40 [04:50<29:30, 50.59s/it]
 BS_ = 300 | lr_ = 1e-05 | n_epoch = 50| HR = 0.1518987341772152 | MRR = 0.0874355368026254 
 15%|█▌        | 6/40 [06:09<34:08, 60.26s/it]
 BS_ = 200 | lr_ = 1e-05 | n_epoch = 150| HR = 0.1518987341772152 | MRR = 0.08663100261201527 
 18%|█▊        | 7/40 [07:52<40:50, 74.27s/it]
 BS_ = 300 | lr_ = 0.001 | n_epoch = 200| HR = 0.339662447257384 | MRR = 0.3096217600964436 
 20%|██        | 8/40 [09:35<44:33, 83.53s/it]


In [ ]:
#Best_param; BS_ = 100 | lr_ = 0.001 | n_epoch = 200 | HR = 0.3459915611814346 | MRR = 0.31348452883263006

In [204]:
##best_BS,best_lr,best_epoch,best_hr,best_mrr = bestModelInfo
print("batch_size     :",100)
print("learning_rate  :",0.001)
print("n_epochs       :",200)
print("best_hr        :",0.34599)   
print("best_mrr       :",0.31348) 

batch_size     : 100
learning_rate  : 0.001
n_epochs       : 200
best_hr        : 0.34599
best_mrr       : 0.31348


In [222]:
#                             #tuning_dim(train_Mat,holdout_df,lr,batch_size,n_epochs,pdim_list):
# pdim_list = [[64,128],[256,512],[100,300],[200,600],[300,900]]
# best_dims,best_hr,best_mrr = tuning_dim(TrainDFRatMat_2,holdout_df2,pdim_list,lr=0.001,batch_size=100,n_epochs=200)

In [219]:
                            #tuning_dim(train_Mat,holdout_df,lr,batch_size,n_epochs,pdim_list):
pdim_list = [[64,128],[256,512],[100,300],[200,600],[300,900]]
best_dims,best_hr,best_mrr = tuning_dim(TrainDFRatMat_2,holdout_df2,pdim_list,lr=0.001,batch_size=200,n_epochs=100)

 20%|██        | 1/5 [01:10<04:40, 70.01s/it]
 p_dim = [64, 128] ,HR = 0.339662447257384 , MRR = 0.30888085191882664 
 40%|████      | 2/5 [02:28<03:45, 75.09s/it]
 p_dim = [256, 512] ,HR = 0.34810126582278483 , MRR = 0.3114593798138102 
 60%|██████    | 3/5 [03:42<02:29, 74.60s/it]
 p_dim = [100, 300] ,HR = 0.33544303797468356 , MRR = 0.30983273056057864 
 80%|████████  | 4/5 [05:03<01:17, 77.25s/it]
 p_dim = [200, 600] ,HR = 0.350210970464135 , MRR = 0.31551470095773887 
100%|██████████| 5/5 [06:39<00:00, 79.91s/it]
 p_dim = [300, 900] ,HR = 0.3333333333333333 , MRR = 0.31237944544906565 

 Best_param; p_dim = [200, 600],HR = 0.350210970464135 ,MRR = 0.31551470095773887



In [227]:
rows_i = newAMZB_DF['Updated_UserID'].nunique()
cols_i = newAMZB_DF['Updated_ItemID'].nunique()

UserItem_MATList = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList))   
UserItem_MATList

10


[<22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 40228 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 41330 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 42432 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 43525 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 44665 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 45752 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 47034 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 48112 stored elements in Compressed Sparse Row format>,
 <22557x

In [239]:
#lr=0.001,batch_size=200,n_epochs=100
batch_size = 200
anneal_cap = 0.2
update_count = 0
n_epochs = 100
constant_anneal = False

total_anneal_steps = getTot_annealSTeps(TrainDFRatMat_2,batch_size,n_epochs,anneal_cap)
total_anneal_steps

38800.0

In [240]:
I = newAMZB_DF['Updated_ItemID'].nunique()

q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

VAEmodel_bty = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)

init_weights(VAEmodel_bty)
VAEmodel_bty.to(device)
optimizer = torch.optim.AdamW(VAEmodel_bty.parameters(), lr=1e-3, weight_decay=0.0)  
VAEmodel_bty 

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=8109, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=8109, bias=True)
    )
  )
)

In [242]:
                                       #Allsteps_predEval(model,UserItemMatList,holdout_DFList,n_epoch)
hr10_List, mrr10_List, ALLUsers_Pred  = Allsteps_predEval(VAEmodel_bty,UserItem_MATList,newHoldout_list,n_epochs) 
print(hr10_List)
print(mrr10_List) 

1it [01:47, 107.98s/it]Number of hits:  164
Total Num of users:  474
Recommendation HitRate:  0.3459915611814346
| HitRate  0.346 | Mrr 0.306 | 
2it [03:35, 107.75s/it]Number of hits:  168
Total Num of users:  433
Recommendation HitRate:  0.38799076212471134
| HitRate  0.388 | Mrr 0.366 | 
3it [05:23, 107.68s/it]Number of hits:  192
Total Num of users:  465
Recommendation HitRate:  0.4129032258064516
| HitRate  0.413 | Mrr 0.365 | 
4it [07:10, 107.37s/it]Number of hits:  181
Total Num of users:  478
Recommendation HitRate:  0.3786610878661088
| HitRate  0.379 | Mrr 0.345 | 
5it [08:58, 107.81s/it]Number of hits:  169
Total Num of users:  478
Recommendation HitRate:  0.35355648535564854
| HitRate  0.354 | Mrr 0.319 | 
6it [10:47, 108.14s/it]Number of hits:  160
Total Num of users:  508
Recommendation HitRate:  0.31496062992125984
| HitRate  0.315 | Mrr 0.279 | 
7it [12:36, 108.31s/it]Number of hits:  246
Total Num of users:  612
Recommendation HitRate:  0.4019607843137255
| HitRate  0.4

In [243]:
hr10_List

[0.3459915611814346,
 0.38799076212471134,
 0.4129032258064516,
 0.3786610878661088,
 0.35355648535564854,
 0.31496062992125984,
 0.4019607843137255,
 0.3823038397328882,
 0.3824626865671642,
 0.26823529411764707]

In [244]:
mrr10_List

[0.30574559641015336,
 0.36602148172586974,
 0.3653328213005632,
 0.3454232250780368,
 0.3186624161519559,
 0.27873703287089113,
 0.27096625687311965,
 0.31361462225402126,
 0.33429208126036486,
 0.2043183940242764]

In [245]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_VAE_AllPRED_new.pkl', 'wb') as f: 
     pickle.dump(ALLUsers_Pred, f)

In [246]:
ALLUsers_Pred[:2]

[array([[  71,  116,    1, ..., 3491, 1191,  191],
        [  62,    9,   43, ...,  297,  692, 4099],
        [   9,    0,   43, ...,  439,    3, 4099],
        ...,
        [  74,   67,  107, ...,  692,  842,   24],
        [  74,   67,  107, ...,  692,  842,   24],
        [  74,   67,  107, ...,  692,  842,   24]]),
 array([[  71,  116,    1, ...,    2, 1385, 1191],
        [  62,   43, 4099, ...,   71,    2,  116],
        [   0,    9, 4099, ...,  231,   60, 5086],
        ...,
        [  74,   67,  107, ...,  692,  842,   25],
        [  74,   67,  107, ...,  692,  842,   25],
        [  74,   67,  107, ...,  692,  842,   25]])]

In [247]:
Cov_List = AllSteps_Coverage_Ratio(newAMZB_DF,ALLUsers_Pred,'Updated_ItemID')
Cov_List 

[0.743988,
 0.768652,
 0.809841,
 0.832655,
 0.860155,
 0.886546,
 0.912566,
 0.924282,
 0.93982,
 0.958318]

In [248]:
AllSteps = list(range(1,10))
print(len(AllSteps))
AllSteps

9


[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [249]:
UpdtPSICorr_ = Updt_getAll_AvgCorr(ALLUsers_Pred,newUserItem_list,AllSteps,'Updated_UserID')  #,
UpdtPSICorr_

array([array([0.62738397, 0.59056136, 0.41874099, ..., 0.06677266, 0.30787632,
              0.24647471])                                                    ,
       array([0.4357129 , 0.72372723, 0.6405868 , ..., 0.36369515, 0.02499653,
              0.03534858])                                                    ,
       array([0.50863567, 0.58102174, 0.42600464, ..., 0.        , 0.14585112,
              0.27357432])                                                    ,
       array([0.32477789, 0.62112893, 0.46158416, ..., 0.21979838, 0.32513465,
              0.08880366])                                                    ,
       array([0.55307733, 0.33689549, 0.37576887, ..., 0.39619786, 0.02928462,
              0.18684676])                                                    ,
       array([0.46462943, 0.65548951, 0.34714364, ..., 0.31980331, 0.17289051,
              0.27621682])                                                    ,
       array([0.54949092, 0.42987214, 0.231192

In [1]:
print(UpdtPSICorr_.shape)
for df in UpdtPSICorr_:
    print(df.mean())  

NameError: name 'UpdtPSICorr_' is not defined

In [ ]:
[0.3277198714275054
0.26332006122461243
0.28246567567737907
0.29588632270231247
0.3172873706719801
0.301323636758813
0.29593038382611475
0.4263375327221993
0.3851709408829141

# Split TrainTest

In [25]:
valid_trn, valid_tst = split_train_test(valid_df, test_prop=0.3)
print(valid_trn.shape, valid_tst.shape) 

0 users sampled
250 users sampled
500 users sampled
750 users sampled
1000 users sampled
1250 users sampled
1500 users sampled
1750 users sampled
2000 users sampled
2250 users sampled
(4916, 4) (165, 4)


In [26]:
test_train, test_tst = split_train_test(test_df, test_prop=0.3)
print(test_train.shape, test_tst.shape)   ##

0 users sampled
250 users sampled
500 users sampled
750 users sampled
1000 users sampled
1250 users sampled
1500 users sampled
1750 users sampled
2000 users sampled
2250 users sampled
(4868, 4) (152, 4)


In [27]:
Train_DF   =    numerize_data(train_df,user2id,item2id,'userId','productId')
ValidTrain_DF = numerize_data(valid_trn,user2id,item2id,'userId','productId')
ValidTest_DF = numerize_data(valid_tst,user2id,item2id,'userId','productId')
TestTrain_DF = numerize_data(test_train,user2id,item2id,'userId','productId')
TestTest_DF =  numerize_data(test_tst,user2id,item2id,'userId','productId')

print("Training Data: ",Train_DF.shape)
print()
print("Validaion DF: ", ValidTrain_DF.shape,ValidTest_DF.shape)
print("Test DF:      ", TestTrain_DF.shape,TestTest_DF.shape)

Training Data:  (40458, 2)

Validaion DF:  (4916, 2) (165, 2)
Test DF:       (4868, 2) (152, 2)


# Convert to RatMat

In [28]:
n_items = len(train_Items)                            #train_Items = unique_sid
TrainDF_RatMat = get_TrainRatMat(Train_DF,'userId','productId',n_items)
ValidTrain_RatMat, ValidTest_RatMat = getTrainTest_RatMat(ValidTrain_DF, ValidTest_DF,'userId','productId',n_items)
TestTrain_RatMat, TestTest_RatMat = getTrainTest_RatMat(TestTrain_DF, TestTest_DF,'userId','productId',n_items)

TrainDF_RatMat

<22799x7400 sparse matrix of type '<class 'numpy.float64'>'
	with 40064 stored elements in Compressed Sparse Row format>

In [ ]:
ValidTrain_RatMat

<22773x7400 sparse matrix of type '<class 'numpy.float64'>'
	with 4861 stored elements in Compressed Sparse Row format>

In [ ]:
ValidTest_RatMat

<22773x7400 sparse matrix of type '<class 'numpy.float64'>'
	with 165 stored elements in Compressed Sparse Row format>

In [ ]:
TestTrain_RatMat

<22759x7400 sparse matrix of type '<class 'numpy.float64'>'
	with 4820 stored elements in Compressed Sparse Row format>

In [ ]:
TestTest_RatMat

<22759x7400 sparse matrix of type '<class 'numpy.float64'>'
	with 152 stored elements in Compressed Sparse Row format>

# Tuning

In [ ]:
# def early_stop(curr_value, best_value, stop_step, patience, score_fn='metric'):
#     if (score_fn == "loss" and curr_value <= best_value) or (score_fn == "metric" and curr_value >= best_value):
#         stop_step, best_value = 0, curr_value
#     else:
#         stop_step += 1
#     if stop_step >= patience:
#         print("Early stopping triggered. patience: {} log:{}".format(patience, best_value))
#         stop = True
#     else:
#         stop = False
#     return best_value, stop_step, stop 

# def tuning_2(model,train_set,validtrain_set,validtest_set,als_param_grid,n_epochs,model_name,save_results= True):
#     update_count = 0
#     best_score = 0     ##based on MetricEval..
#     stop_step = 0
#     update_count = 0
#     stop = False

#     for i in tqdm(range(len(als_param_grid))):
#         batch_size, lr_, weightdecay_ = list(als_param_grid)[i]
#         total_anneal_steps = getTot_annealSTeps(train_set,batch_size,n_epochs,anneal_cap)
#         Optimizer_ = torch.optim.AdamW(model.parameters(), lr=lr_, weight_decay=weightdecay_)
#         print('batch_size = {} ,learning_rate = {}, weight_decay = {} '.format(batch_size,lr_,weightdecay_))
#         model_2 = model
#         for epoch in range(n_epochs):            
#             train_step_2(model_2, Optimizer_, train_set,total_anneal_steps,epoch)
#             val_loss, NDCG100_ = eval_step_2(model_2,validtrain_set,validtest_set)
#             curr_score = NDCG100_
#             best_score,stop_step,stop = early_stop(curr_score,best_score,stop_step,
#                                            patience=25,score_fn='metric')
#             if stop:
#                 break
#             if (stop_step == 0) & (save_results):
#                best_epoch = epoch
#                best_BS  = batch_size
#                best_lr_ = lr_
#                best_weightdecay_ = weightdecay_
#                best_NGCG100 = NDCG100_
#                torch.save(model_2.state_dict(),model_name+".pth")
#                #print('NDCG100_ = {} '.format(best_NGCG100))

#         print('NDCG100_ = {} '.format(best_NGCG100))

#     print('\n Best parameters; batch_size= {}, learning_rate = {}, weight_decay = {}, NDCG100_ = {}, best_epoch = {}'.format(best_BS,
#                                                                             best_lr_,best_weightdecay_,best_NGCG100,best_epoch))
#     return best_BS, best_lr_, best_weightdecay_,best_NGCG100, best_epoch  

In [48]:
# als_params = dict(batch_size = [50,100,200,300,500],
#                   lr = [1e-7,1e-5,1e-3,1e-1],
#                   weight_decay= [0.,0.1,0.2,0.3])

# als_param_grid, als_param_names = random_grid(als_params, n=50)
# print(als_param_names)
# len(als_param_grid)                 

('batch_size', 'lr', 'weight_decay')


50

In [49]:
# for i in range(10):
#     batch_size, lr, weight_decay = list(als_param_grid)[i]  
#     print(batch_size, lr, weight_decay)

200 1e-05 0.1
50 1e-05 0.2
200 0.001 0.2
300 1e-07 0.0
500 0.1 0.3
300 0.1 0.3
50 1e-07 0.0
200 0.1 0.2
300 0.001 0.2
200 1e-05 0.0


In [50]:
n_items = len(train_Items)   
I = n_items    ##I is the total number of items in the train date
q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

In [51]:
                    #p_dims,q_dims,dropout_enc,dropout_dec
VAEmodel = MultiVAE(p_dims,q_dims,dropout_enc,dropout_dec)
init_weights(VAEmodel)
VAEmodel.to(device)

MultiVAE(
  (encode): Encoder(
    (q_layers): Sequential(
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_0): Linear(in_features=7400, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=400, bias=True)
    )
  )
  (decode): Decoder(
    (p_layers): Sequential(
      (dropout_0): Dropout(p=0.0, inplace=False)
      (linear_0): Linear(in_features=200, out_features=600, bias=True)
      (dropout_1): Dropout(p=0.0, inplace=False)
      (linear_1): Linear(in_features=600, out_features=7400, bias=True)
    )
  )
)

In [52]:
device

device(type='cuda')

In [53]:
anneal_epochs = None  
anneal_cap = 0.2
constant_anneal = False
update_count = 0

#tuning_2(model,train_set,validtrain_set,validtest_set,als_param_grid,200,'AMZBModel',save_results= True)
bestModel_info =  tuning_2(VAEmodel,TrainDF_RatMat,ValidTrain_RatMat,ValidTest_RatMat,als_param_grid,200,'AMZBModel',save_results= True)

  0%|          | 0/50 [00:00<?, ?it/s]

batch_size = 200 ,learning_rate = 1e-05, weight_decay = 0.1 


  2%|▏         | 1/50 [04:57<4:02:33, 297.00s/it]

Early stopping triggered. patience: 25 log:0.3178213020949254
NDCG100_ = 0.3178213020949254 
batch_size = 50 ,learning_rate = 1e-05, weight_decay = 0.2 


  4%|▍         | 2/50 [05:01<1:39:44, 124.68s/it]

...

Early stopping triggered. patience: 25 log:0.6664325618825107
NDCG100_ = 0.6664325618825107 
batch_size = 300 ,learning_rate = 0.001, weight_decay = 0.3 


 98%|█████████▊| 49/50 [09:38<00:03,  3.16s/it]

Early stopping triggered. patience: 25 log:0.6664325618825107
NDCG100_ = 0.6664325618825107 
batch_size = 100 ,learning_rate = 1e-05, weight_decay = 0.2 


100%|██████████| 50/50 [09:41<00:00, 11.63s/it]

Early stopping triggered. patience: 25 log:0.6664325618825107
NDCG100_ = 0.6664325618825107 

 Best parameters; batch_size= 200, learning_rate = 0.001, weight_decay = 0.2, NDCG100_ = 0.6664325618825107, best_epoch = 4


In [54]:
best_BS, best_lr_, best_weightdecay_,best_NGCG100, best_epoch =  bestModel_info
print("batch_size     :",best_BS)
print("learning_rate  :",best_lr_)
print("weight_decay   :",best_weightdecay_)
print("best_NGCG100   :",best_NGCG100)        
print("best_epoch      :",best_epoch)

batch_size     : 200
learning_rate  : 0.001
weight_decay   : 0.2
best_NGCG100   : 0.6664325618825107
best_epoch      : 4


In [ ]:
#/home/oolaleke/incremental_PSI/Notebooks/AMZBModel.pth

In [55]:
# Run on test data with best model
#model.load_state_dict(torch.load(model_name + ".pt")))
VAEmodel.load_state_dict(torch.load('AMZBModel.pth'))
test_loss,tst_ndgc100_ = eval_step_2(VAEmodel,TestTrain_RatMat,TestTest_RatMat)
tst_ndgc100_

0.6940390856562167

# get Prediction

In [38]:
X_out_1 = get_ModelOut(X_tr1,VAEmodel)
print(X_out_1.shape)
X_out_1

NameError: name 'X_tr1' is not defined

In [ ]:
top10_sample  = get_TopNPred(X_out_1,k=10)
top10_sample

array([[ 56,  64, 200, ..., 273,  90,  91],
       [ 56,  64, 200, ..., 273,  90,  91],
       [ 56,  64, 200, ..., 273,  90,  91],
       ...,
       [ 56,  64, 200, ..., 273,  90,  91],
       [ 56,  64, 200, ..., 273,  90,  91],
       [ 56,  64, 200, ..., 273,  90,  91]])

In [ ]:
for i in range(10): 
    print(top10_sample[i])